In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
!gdown --id 1xIuSYMrd3oBZb5U28VbV-oqT5DQd-HtW

Downloading...
From: https://drive.google.com/uc?id=1xIuSYMrd3oBZb5U28VbV-oqT5DQd-HtW
To: /content/Social_Network_Ads.csv
100% 4.91k/4.91k [00:00<00:00, 12.7MB/s]


In [ ]:
data =  pd.read_csv('Social_Network_Ads.csv')
data

,Age,EstimatedSalary,Purchased
0,19,19000,0
1,35,20000,0
2,26,43000,0
3,27,57000,0
4,19,76000,0
...,...,...,...
395,46,41000,1
396,51,23000,1
397,50,20000,1
398,36,33000,0


In [ ]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, 2].values

# Áp dụng Grid Search cho CNN và SVM

## B0: SVM có những siêu tham số nào? Miền giá trị của từng tham số.

## Có 3 siêu tham số:
## - Tham số C: float, default=1.0

## - gamma: {‘scale’, ‘auto’} or float, default=’scale’

## - kernel: {‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’

## B1: Chia dataset: 
## train : val : test = 6 : 2 : 2

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=0)   # 8 : 2

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=1)   # 0.25 x 0.8 = 0.2

## B2: Chuẩn hoá, one-hot encoding

In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

## SVM

In [ ]:
# SVM 

model = svm.SVC()
model.fit(X_train, Y_train)

pre = model.predict(X_val)
print(classification_report(Y_val, pre))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        51
           1       0.36      1.00      0.53        29

    accuracy                           0.36        80
   macro avg       0.18      0.50      0.27        80
weighted avg       0.13      0.36      0.19        80



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
C_star = -1
gamma_star = -1
kernel_star = ''
score = -1

for i in range(5): # for C
    for j in range(10): # for gamma
        for _kernel in ['linear', 'poly', 'rbf', 'sigmoid']: # for kernal
            _C = 2**i
            _gamma = 2**(-j)
            model = svm.SVC(C=_C, kernel=_kernel, gamma=_gamma)
            model.fit(X_train, Y_train)
            new_score = model.score(X_val, Y_val)
            print('C = {}; gamma = {}; kernel = {}; score = {}; new_score = {}'.format(_C, _gamma, _kernel, score, new_score))
            if new_score > score:
                C_star = _C
                gamma_star = _gamma
                kernal_star = _kernel
                score = new_score

C = 1; gamma = 1; kernel = linear; score = -1; new_score = 0.3625
C = 1; gamma = 1; kernel = poly; score = 0.3625; new_score = 0.3625
C = 1; gamma = 1; kernel = rbf; score = 0.3625; new_score = 0.3625
C = 1; gamma = 1; kernel = sigmoid; score = 0.3625; new_score = 0.6375
C = 1; gamma = 0.5; kernel = linear; score = 0.6375; new_score = 0.3625
C = 1; gamma = 0.5; kernel = poly; score = 0.6375; new_score = 0.3625
C = 1; gamma = 0.5; kernel = rbf; score = 0.6375; new_score = 0.3625
C = 1; gamma = 0.5; kernel = sigmoid; score = 0.6375; new_score = 0.6375
C = 1; gamma = 0.25; kernel = linear; score = 0.6375; new_score = 0.3625
C = 1; gamma = 0.25; kernel = poly; score = 0.6375; new_score = 0.3625
C = 1; gamma = 0.25; kernel = rbf; score = 0.6375; new_score = 0.3625
C = 1; gamma = 0.25; kernel = sigmoid; score = 0.6375; new_score = 0.6375
C = 1; gamma = 0.125; kernel = linear; score = 0.6375; new_score = 0.3625
C = 1; gamma = 0.125; kernel = poly; score = 0.6375; new_score = 0.3625
C = 1; gam

In [ ]:
print("C* = ", C_star)
print("Kernal* = ", kernal_star)
print("gamma* = ", gamma_star)
print("Score on valid set: ", score)

C* =  1
Kernal* =  sigmoid
gamma* =  1
Score on valid set:  0.6375


## B4: Tính Accuracy của test

In [ ]:
model_2 = svm.SVC(C=C_star, kernel=kernal_star, gamma=gamma_star)
model_2.fit(X_train, Y_train)

print("Accuracy on test set: ",model_2.score(X_test, Y_test))

Accuracy on test set:  0.725
